# RAG のための前処理

指定ディレクトリ内のすべての .txt ファイルを読み込み、NLTK を使ってテキストを文ごとに分割し、指定した文数ごとにチャンク化した上で、各チャンクの埋め込みを生成して出力します。

split_text_into_chunks 関数
NLTK の sent_tokenize を使用してテキストを文単位に分割し、max_sentences 毎に結合してチャンク化しています。

get_embedding 関数
最新の OpenAI API では、input パラメータにテキストのリストを渡す必要があるため、input=[text] としています。
生成された応答から、最初のチャンクの埋め込みを取り出して返します。

main 関数
指定ディレクトリ内の全 .txt ファイルを読み込み、各ファイルのテキストをチャンクに分割し、各チャンクに対して埋め込みを生成、最初の5次元だけを出力しています。

このコードを実行すれば、まずは小さなサンプルテキストを使った前処理と埋め込み生成の流れを体験できます。
その後、生成した埋め込みを Pinecone や Weaviate などのベクトルデータベースに保存するなど、RAG システムの次のステップに進むことができます。

### テキストの分割・前処理
- 長い文章は、意味のまとまりごとに分割する必要があります。

- たとえば、「文章を文単位に分割し、さらにいくつかの文ごとにまとめてチャンクを作る」方法が考えられます。

# 🔧 これまでのステップ：RAG構築の準備プロセス
### 1. Pineconeのセットアップ
- PineconeのWebコンソールにログインし、新しいプロジェクト・インデックス（hamster-embeddings）を作成。

- 設定内容：
    - Dimension: 1536（OpenAIのtext-embedding-ada-002に対応）
    - Metric: cosine
    - Type: Dense
    - Capacity Mode: Serverless
    - Cloud: AWS、Region: us-east-1

### 2. Docker環境の構築
- Jupyter環境でNotebookを使ってRAG処理を行うため、DockerでPython+Jupyterの環境を構築。

- requirements.txtに以下の主要ライブラリを指定：
    - openai>=1.0.0
    - pinecone-client==2.2.4
    - httpx
    - nltk
    - jupyter

- Dockerfileでは追加で build-essential, ca-certificates, curl などもインストールし、通信や証明書の問題を回避。

### 3. テキストデータの分割とベクトル化
- senarios/ ディレクトリ内の .txt ファイルを読み込み、NLTKの sent_tokenize を使って文単位でチャンクに分割。

- OpenAIの text-embedding-ada-002 モデルを使って、各チャンクの文章を1536次元の埋め込みベクトルに変換。

### 4. Pineconeへのアップロード（Upsert）
- pinecone.Index.upsert() により、ベクトルとメタデータ（元のテキスト）をPineconeに保存。

- 実行成功後、Pineconeのダッシュボードで Record Count: 11 を確認（→成功！🎉）

### Dockerコンテナ起動方法
- Docker Image を構築する。
    - ❯ docker build -t hamster-chunk-text .
- Docker Container を使う
    - ❯ docker run -it -p 8888:8888 --name hamster-chunk-text \
  --env-file .env \
  -v $(pwd)/senarios:/app/senarios \
  hamster-chunk-text

### VSCodeの中で、コンテナを実行する（カーネルに指定する）方法
- ターミナル上でDocker Container をrunしたあとに出てくる以下のログをコピーしておく。
    - [I 2025-05-02 08:59:45.108 ServerApp] http://127.0.0.1:8888/tree
- VSCode で、この .ipynb を開き、

    Notebook 上部に表示される カーネル選択バーで「既存のJupyterサーバーに接続」を選択し、

    先ほどコピーした http://127.0.0.1:8888/... を貼り付ける。

In [1]:
import socket

try:
    print("🔍 解決結果:", socket.gethostbyname("hamster-embeddings-4ak3maz.svc.us-east-1-aws.pinecone.io"))
except Exception as e:
    print("❌ 名前解決できませんでした:", e)


🔍 解決結果: 52.6.114.50


In [2]:
print('hello, http://127.0.0.1:8888/tree')

hello, http://127.0.0.1:8888/tree


In [3]:
from openai import OpenAI
from pinecone import Pinecone
from nltk.tokenize import sent_tokenize
import nltk
import os
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [4]:
# OpenAI クライアントの初期化
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Pinecone クライアントの初期化
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

# インデックスに接続
index = pc.Index("hamster-embeddings")

# コンテナのディレクトリ
directory = "./senarios"

In [5]:
# --- ユーティリティ関数 ---
def split_text_into_chunks(text, max_sentences=2):
    sentences = sent_tokenize(text)
    return [" ".join(sentences[i:i+max_sentences]) for i in range(0, len(sentences), max_sentences)]

def get_embedding(text):
    res = client.embeddings.create(input=[text], model="text-embedding-ada-002")
    return res.data[0].embedding

In [6]:
# チャンクと埋め込みの生成（これで chunks と embeddings が作られます）
chunks = []

for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        with open(os.path.join(directory, filename), 'r', encoding='utf-8') as f:
            text = f.read()
        chunks.extend(split_text_into_chunks(text))

# 埋め込みの生成
embeddings = [get_embedding(chunk) for chunk in chunks]


In [7]:
# Pinecone へアップロード（今のエラーが出たセル）

vectors = []
for i, embedding in enumerate(embeddings):
    vector_id = f"chunk{i+1}"
    metadata = {"text": chunks[i]}
    vectors.append((vector_id, embedding, metadata))

# ベクトルを Pinecone にアップロード
index.upsert(vectors=vectors)

print("✅ Embeddings uploaded successfully to Pinecone!")


✅ Embeddings uploaded successfully to Pinecone!


# 🧠 次に目指すステップ
以下のような処理を実装していくことで、RAGの完全動作へと進化できます。

## ✅ 今後のステップ案
### 1.検索（ベクトル検索）機能の実装

- クエリテキストを入力し、その埋め込みベクトルを生成。
- Pineconeで類似ベクトルを検索し、最も近いテキストを取得。

### 2.OpenAIへのプロンプト生成

- 検索結果から得たチャンクを組み合わせて「コンテキスト付きプロンプト」を構築。
- ChatGPTにそのプロンプトを送信し、回答を得る。

### 3.ユーザーインターフェース（例：Streamlit or Flask）
- ユーザーがクエリを入力し、検索〜応答生成を体験できるUIを構築。

### 4.チャットの履歴保存や拡張（任意）
- 会話履歴を保存、マルチターンの対話なども視野に。


素晴らしい進捗です🎉  
GitHubへのプッシュ完了＆Docker環境構築もバッチリ整って、**RAG（Retrieval-Augmented Generation）基盤の準備完了！**  
では、いよいよ **次のステップ「RAGの完成形に向けた構築フェーズ」** に進みましょう！

---

## 🔜 次のステップ：RAGの「検索→生成」パイプラインの実装

以下のような構成に段階的に仕上げていきます：

### 🧠 1. クエリ → 埋め込み生成
- ユーザーの質問（例：`「ハムスターが夜にうるさい時の対処法」`）を `text-embedding-ada-002` でベクトル化

### 📚 2. Pinecone で類似検索
- Pinecone に保存済みのチャンクベクトル（`senarios/*.txt`由来）と照合して、類似度の高いチャンクを複数件取得（例：Top5）

### 🧾 3. コンテキスト付きでChatGPTへプロンプト送信
- 検索結果を `context` として埋め込んだうえで ChatGPT API に問い合わせ
- プロンプトテンプレート例：
  ```
  あなたはハムスター飼育の専門家です。
  以下の情報を参考にして質問に答えてください。

  コンテキスト:
  {context}

  質問:
  {question}
  ```

### 💬 4. ChatGPTからの自然な回答をユーザーに提示
- 最終的にはこれを**WebアプリやチャットUIに組み込む**ところまでを視野に

---

## ✅ 最初にやること

次のNotebookかスクリプトを追加して、検索→生成の流れを実装しましょう。以下のファイル名を提案します：

```bash
rag_query.ipynb
```

この中に、以下の処理を入れていきます：

### 🎯 rag_query.ipynb の構成案

```python
# 1. ライブラリ読み込み
import os
from openai import OpenAI
import pinecone
from dotenv import load_dotenv
import numpy as np

# 2. 初期設定
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
pinecone.init(api_key=os.getenv("PINECONE_API_KEY"), environment="us-east-1-aws")
index = pinecone.Index("hamster-embeddings")

# 3. 質問入力
question = "ハムスターが夜にうるさい時の対処法は？"

# 4. 質問をベクトル化
embedding = client.embeddings.create(
    model="text-embedding-ada-002",
    input=question
).data[0].embedding

# 5. Pinecone検索
response = index.query(vector=embedding, top_k=5, include_metadata=True)
contexts = [match["metadata"]["text"] for match in response["matches"]]

# 6. プロンプト作成
prompt = f"""あなたはハムスター飼育の専門家です。
以下の情報を参考にして質問に答えてください。

コンテキスト:
{chr(10).join(contexts)}

質問:
{question}
"""

# 7. GPT呼び出し
chat_response = client.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user", "content": prompt}]
)

# 8. 出力
print(chat_response.choices[0].message.content)
```

---

## 📁 お願い

- このコードを `.ipynb` で試したいですか？それとも `.py` スクリプトとして動かしたいですか？
- また、**質問はユーザー入力にしたい？**（`input()`やWebUIを使う）かも教えてください！

---

一緒に「ハムスターチャットボット」完成を目指しましょう〜🐹✨  
ご希望の実装スタイルを教えてくれれば、次のファイルを一緒に作ります！